In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
import warnings
from nltk.stem.snowball import SnowballStemmer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import pandas_profiling as pp
import seaborn as sns
import matplotlib as plt


In [2]:
#Preprocessing :
"""
1- removing spaces that are not required
2- converting upper to lower
3- removing stopwords in english
4- combing all labels into one
"""


df = pd.read_csv("blogtext.csv",nrows=100000)
pattern = "[^\w ]"
df.text = df.text.apply(lambda s : re.sub(pattern,"",s))


df.text = df.text.apply(lambda s: s.lower())
df.text = df.text.apply(lambda s: s.strip())


stopwords=set(stopwords.words('english'))
df.text = df.text.apply(lambda t: ' '.join([words for words in t.split() if words not in stopwords]))
df.drop(['id','date'], axis=1,inplace=True)
                                   
df['labels'] = df.apply(lambda col : [col['gender'],col['age']], axis=1)
df.drop(columns=['gender','age','topic','sign'], axis=1, inplace=True)                                  
                                
                                  

In [3]:
X= df.text
y = df.labels
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=2,test_size = 0.2)

In [4]:
count_vector = CountVectorizer(ngram_range=(1,2))
count_vector.fit(X_train)

X_train_count = count_vector.transform(X_train)
x_test_count = count_vector.transform(X_test)


lcs=dict()

for labels in df.labels.values:
    for label in labels:
        if label in lcs:
            lcs[str(label)]+=1
        else:
            lcs[str(label)]=1
            
            
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(lcs.keys()))

y_train = binarizer.fit_transform(y_train)
y_test = binarizer.transform(y_test)

model=LogisticRegression(solver='lbfgs', max_iter=100)
model=OneVsRestClassifier(model)
model.fit(X_train_count,y_train)

C:\Users\MSI\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:860: UserWarning: unknown class(es) [13, 14, 15, 16, 17, 23, 24, 25, 26, 27, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 1 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 2 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 3 is present in all training examples.


OneVsRestClassifier(estimator=LogisticRegression())

In [6]:
Ypred=model.predict(x_test_count)

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def metrics(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: Micro', f1_score(Ytest, Ypred, average='micro'))
    print('precision score', average_precision_score(Ytest, Ypred, average='micro'))
    print('recall score', recall_score(Ytest, Ypred, average='micro'))
    
    

metrics(y_test,Ypred)   

Accuracy score:  0.64865
F1 score: Micro 0.7120783818645882
precision score 0.5244990842741549
recall score 0.64865


<80000x4517090 sparse matrix of type '<class 'numpy.int64'>'
	with 13894625 stored elements in Compressed Sparse Row format>

In [28]:
label_counts

{'male': 53358,
 '15': 1,
 '33': 1,
 'female': 46642,
 '14': 1,
 '25': 1,
 '17': 1,
 '23': 1,
 '37': 1,
 '26': 1,
 '24': 1,
 '27': 1,
 '45': 1,
 '34': 1,
 '41': 1,
 '44': 1,
 '16': 1,
 '39': 1,
 '35': 1,
 '36': 1,
 '46': 1,
 '42': 1,
 '13': 1,
 '38': 1,
 '43': 1,
 '40': 1,
 '47': 1,
 '48': 1}

C:\Users\MSI\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:860: UserWarning: unknown class(es) [13, 14, 15, 16, 17, 23, 24, 25, 26, 27, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 1 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 2 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 3 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 4 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\MSI\anaconda3\li

OneVsRestClassifier(estimator=LogisticRegression())